<a href="https://colab.research.google.com/github/satyajitghana/TSAI-DeepNLP-END2.0/blob/main/12_AttentionIsAllYouNeed/Attention_Is_All_You_Need_(TPU).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Attention Is All You Need !

In [1]:
! pip3 install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl

     |████████████████████████████████| 149.9 MB 70 kB/s 
     |████████████████████████████████| 57 kB 2.6 MB/s 
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 1.12.8
    Uninstalling google-api-python-client-1.12.8:
      Successfully uninstalled google-api-python-client-1.12.8
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
earthengine-api 0.1.272 requires google-api-python-client<2,>=1.12.1, but you have google-api-python-client 1.8.0 which is incompatible.


In [2]:
! pip3 install git+https://github.com/extensive-nlp/ttc_nlp --quiet

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 913 kB 5.3 MB/s 
     |████████████████████████████████| 74 kB 2.5 MB/s 
     |████████████████████████████████| 234 kB 58.2 MB/s 
     |████████████████████████████████| 22.3 MB 2.0 MB/s 
     |████████████████████████████████| 6.4 MB 31.7 MB/s 
     |████████████████████████████████| 76 kB 4.3 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 112 kB 42.8 MB/s 
     |████████████████████████████████| 636 kB 34.1 MB/s 
     |████████████████████████████████| 10.6 MB 37.2 MB/s 
     |████████████████████████████████| 118 kB 57.6 MB/s 
     |████████████████████████████████| 829 kB 47.4 MB/s 
     |████████████████████████████████| 1.3 MB 45.4 MB/s 
     |████████████████████████████████| 42 kB 1.3 MB/s 


In [3]:
! python -m spacy download en_core_web_sm --quiet
! python -m spacy download de_core_news_sm --quiet

2021-07-29 17:25:15.604892: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
     |████████████████████████████████| 13.6 MB 71 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
2021-07-29 17:25:24.900708: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
     |████████████████████████████████| 18.8 MB 1.3 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')


## Dataset

In [4]:
import torch
import pytorch_lightning as pl
from torchtext.datasets import Multi30k, IWSLT2016

from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

from torch.utils.data import DataLoader

from torchtext.datasets import Multi30k, IWSLT2016

from typing import *

# helper function to club together sequential operations
def sequential_transforms(*transforms):
    def func(txt_input):
        for transform in transforms:
            txt_input = transform(txt_input)
        return txt_input
    return func

class TTCTranslation(pl.LightningDataModule):

    DATASET_OPTIONS = ['multi30k', 'iwslt2016']
    LANGUAGE_OPTIONS = ['en', 'fr', 'de', 'cs', 'ar']

    # Define special symbols and indices
    UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3
    # Make sure the tokens are in order of their indices to properly insert them in vocab
    SPECIAL_SYMBOLS = ['<unk>', '<pad>', '<bos>', '<eos>']

    def __init__(self, language_pair=('en', 'de'), spacy_language_pair=('en_core_web_sm', 'de_core_news_sm'), dataset='multi30k', batch_size=64, batch_first=True):
        super().__init__()

        assert len(language_pair) == 2 and len(spacy_language_pair), f'tf are you doing? give me a language \"pair\"'
        assert dataset in self.DATASET_OPTIONS, f'{self.DATASET_OPTIONS} are only supported'
        assert all(x in self.LANGUAGE_OPTIONS for x in language_pair), f'{self.LANGUAGE_OPTIONS} are only supported'

        self.batch_size = batch_size
        self.batch_first = batch_first

        self.language_pair = language_pair
        self.src_lang, self.tgt_lang = language_pair
        self.spacy_src_lang, self.spacy_tgt_lang = spacy_language_pair

        if dataset == 'multi30k':
            self.train_dataset = Multi30k(split='train', language_pair=self.language_pair)
            self.val_dataset = Multi30k(split='valid', language_pair=self.language_pair)
            self.test_dataset = Multi30k(split='test', language_pair=self.language_pair)
        elif dataset == 'iwslt2016':
            self.train_dataset = IWSLT2016(split='train', language_pair=self.language_pair)
            self.val_dataset = IWSLT2016(split='valid', language_pair=self.language_pair)
            self.test_dataset = IWSLT2016(split='test', language_pair=self.language_pair)
        
        self.train_dataset, self.val_dataset, self.test_dataset = list(self.train_dataset), list(self.val_dataset), list(self.test_dataset)

        # --- token transform ---

        self.token_transform = {}
        self.token_transform[self.src_lang] = get_tokenizer('spacy', language=self.spacy_src_lang)
        self.token_transform[self.tgt_lang] = get_tokenizer('spacy', language=self.spacy_tgt_lang)

        # --- vocab transform ---
        # helper function to yield list of tokens
        def yield_tokens(data_iter: Iterable, language: str) -> List[str]:
            language_index = {self.src_lang: 0, self.tgt_lang: 1}

            for data_sample in data_iter:
                yield self.token_transform[language](data_sample[language_index[language]])
        
        self.vocab_transform = {}
        for ln in self.language_pair:
            # Create torchtext's Vocab object 
            self.vocab_transform[ln] = build_vocab_from_iterator(yield_tokens(self.train_dataset, ln),
                                                            min_freq=1,
                                                            specials=self.SPECIAL_SYMBOLS,
                                                            special_first=True)

        # Set UNK_IDX as the default index. This index is returned when the token is not found. 
        # If not set, it throws RuntimeError when the queried token is not found in the Vocabulary. 
        for ln in self.language_pair:
            self.vocab_transform[ln].set_default_index(self.UNK_IDX)

        # --- text/tensor transform ---

        # function to add BOS/EOS and create tensor for input sequence indices
        def tensor_transform(token_ids: List[int]):
            return torch.cat((torch.tensor([self.BOS_IDX]), 
                            torch.tensor(token_ids), 
                            torch.tensor([self.EOS_IDX])))

        # src and tgt language text transforms to convert raw strings into tensors indices
        self.text_transform = {}
        for ln in self.language_pair:
            self.text_transform[ln] = sequential_transforms(self.token_transform[ln], #Tokenization
                                                    self.vocab_transform[ln], #Numericalization
                                                    tensor_transform) # Add BOS/EOS and create tensor

    def prepare_data(self):
        # download, split, etc...
        # only called on 1 GPU/TPU in distributed
        pass

    def setup(self, stage):
        # make assignments here (val/train/test split)
        # called on every process in DDP
        pass

    def train_dataloader(self):
        return DataLoader(
            self.train_dataset,
            batch_size=self.batch_size,
            collate_fn=self.collator_fn 
        )

    def val_dataloader(self):
        return DataLoader(
            self.val_dataset,
            batch_size=self.batch_size,
            collate_fn=self.collator_fn
        )

    def test_dataloader(self):
        return DataLoader(
            self.test_dataset,
            batch_size=self.batch_size,
            collate_fn=self.collator_fn
        )

    @property
    def collator_fn(self):
        def wrapper(batch):
            src_batch, tgt_batch = [], []
            for src_sample, tgt_sample in batch:
                src_batch.append(self.text_transform[self.src_lang](src_sample.rstrip("\n")))
                tgt_batch.append(self.text_transform[self.tgt_lang](tgt_sample.rstrip("\n")))

            src_batch = torch.nn.utils.rnn.pad_sequence(src_batch, padding_value=self.PAD_IDX, batch_first=self.batch_first)
            tgt_batch = torch.nn.utils.rnn.pad_sequence(tgt_batch, padding_value=self.PAD_IDX, batch_first=self.batch_first)

            return src_batch, tgt_batch

        return wrapper

    def teardown(self):
        # clean up after fit or test
        # called on every process in DDP
        pass

In [5]:
ttc_translation = TTCTranslation(
    language_pair=('en', 'de'),
    spacy_language_pair=('en_core_web_sm', 'de_core_news_sm'),
    dataset='multi30k',
    batch_size=4
)

training.tar.gz: 100%|██████████| 1.21M/1.21M [00:01<00:00, 1.19MB/s]
validation.tar.gz: 100%|██████████| 46.3k/46.3k [00:00<00:00, 231kB/s]
mmt16_task1_test.tar.gz: 100%|██████████| 43.9k/43.9k [00:00<00:00, 218kB/s]


In [6]:
train_loader = ttc_translation.train_dataloader()

In [7]:
src, tgt = next(iter(train_loader))

In [8]:
src.shape, tgt.shape

(torch.Size([4, 17]), torch.Size([4, 17]))

In [9]:
src_itos = ttc_translation.vocab_transform[ttc_translation.src_lang].get_itos()
tgt_itos = ttc_translation.vocab_transform[ttc_translation.tgt_lang].get_itos()

In [10]:
src[0]

tensor([   2,   20,   26,   16, 1170,  809,   18,   58,   85,  337, 1340,    6,
           3,    1,    1,    1,    1])

In [11]:
tgt[0]

tensor([   2,   22,   86,  258,   32,   88,   23,   95,    8,   17,  113, 7911,
        3210,    5,    3,    1,    1])

In [12]:
' '.join(src_itos[x] for x in src[0])

'<bos> Two young , White males are outside near many bushes . <eos> <pad> <pad> <pad> <pad>'

In [13]:
' '.join(tgt_itos[x] for x in tgt[0])

'<bos> Zwei junge weiße Männer sind im Freien in der Nähe vieler Büsche . <eos> <pad> <pad>'

In [14]:
tgt

tensor([[   2,   22,   86,  258,   32,   88,   23,   95,    8,   17,  113, 7911,
         3210,    5,    3,    1,    1],
        [   2,   85,   32,   11,  848, 2209,   16, 8269,    5,    3,    1,    1,
            1,    1,    1,    1,    1],
        [   2,    6,   70,   28,  220,    8,   16, 6770,   56,  509,    5,    3,
            1,    1,    1,    1,    1],
        [   2,    6,   13,    8,    7,   48,   42,   31,   12,   14,  544,   10,
          699,   16,  249,    5,    3]])

In [15]:
tgt[:, 1:]

tensor([[  22,   86,  258,   32,   88,   23,   95,    8,   17,  113, 7911, 3210,
            5,    3,    1,    1],
        [  85,   32,   11,  848, 2209,   16, 8269,    5,    3,    1,    1,    1,
            1,    1,    1,    1],
        [   6,   70,   28,  220,    8,   16, 6770,   56,  509,    5,    3,    1,
            1,    1,    1,    1],
        [   6,   13,    8,    7,   48,   42,   31,   12,   14,  544,   10,  699,
           16,  249,    5,    3]])

## Model

In [27]:
import torch
import torch.nn as nn


class SelfAttention(nn.Module):
    def __init__(self, embed_size, heads):
        super(SelfAttention, self).__init__()
        self.embed_size = embed_size
        self.heads = heads
        self.head_dim = embed_size // heads

        assert (
            self.head_dim * heads == embed_size
        ), "Embedding size needs to be divisible by heads"

        self.values = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.keys = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.queries = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.fc_out = nn.Linear(heads * self.head_dim, embed_size)

    def forward(self, values, keys, query, mask):
        # Get number of training examples
        N = query.shape[0]

        value_len, key_len, query_len = values.shape[1], keys.shape[1], query.shape[1]

        # Split the embedding into self.heads different pieces
        values = values.reshape(N, value_len, self.heads, self.head_dim)
        keys = keys.reshape(N, key_len, self.heads, self.head_dim)
        query = query.reshape(N, query_len, self.heads, self.head_dim)

        values = self.values(values)  # (N, value_len, heads, head_dim)
        keys = self.keys(keys)  # (N, key_len, heads, head_dim)
        queries = self.queries(query)  # (N, query_len, heads, heads_dim)

        # Einsum does matrix mult. for query*keys for each training example
        # with every other training example, don't be confused by einsum
        # it's just how I like doing matrix multiplication & bmm

        energy = torch.einsum("nqhd,nkhd->nhqk", [queries, keys])
        # queries shape: (N, query_len, heads, heads_dim),
        # keys shape: (N, key_len, heads, heads_dim)
        # energy: (N, heads, query_len, key_len)

        # Mask padded indices so their weights become 0
        if mask is not None:
            energy = energy.masked_fill(mask == 0, float("-1e20"))

        # Normalize energy values similarly to seq2seq + attention
        # so that they sum to 1. Also divide by scaling factor for
        # better stability
        attention = torch.softmax(energy / (self.embed_size ** (1 / 2)), dim=3)
        # attention shape: (N, heads, query_len, key_len)

        out = torch.einsum("nhql,nlhd->nqhd", [attention, values]).reshape(
            N, query_len, self.heads * self.head_dim
        )
        # attention shape: (N, heads, query_len, key_len)
        # values shape: (N, value_len, heads, heads_dim)
        # out after matrix multiply: (N, query_len, heads, head_dim), then
        # we reshape and flatten the last two dimensions.

        out = self.fc_out(out)
        # Linear layer doesn't modify the shape, final shape will be
        # (N, query_len, embed_size)

        return out


class TransformerBlock(nn.Module):
    def __init__(self, embed_size, heads, dropout, forward_expansion):
        super(TransformerBlock, self).__init__()
        self.attention = SelfAttention(embed_size, heads)
        self.norm1 = nn.LayerNorm(embed_size)
        self.norm2 = nn.LayerNorm(embed_size)

        self.feed_forward = nn.Sequential(
            nn.Linear(embed_size, forward_expansion * embed_size),
            nn.ReLU(),
            nn.Linear(forward_expansion * embed_size, embed_size),
        )

        self.dropout = nn.Dropout(dropout)

    def forward(self, value, key, query, mask):
        attention = self.attention(value, key, query, mask)

        # Add skip connection, run through normalization and finally dropout
        x = self.dropout(self.norm1(attention + query))
        forward = self.feed_forward(x)
        out = self.dropout(self.norm2(forward + x))
        return out


class Encoder(nn.Module):
    def __init__(
        self,
        src_vocab_size,
        embed_size,
        num_layers,
        heads,
        forward_expansion,
        dropout,
        max_length,
    ):

        super(Encoder, self).__init__()
        self.embed_size = embed_size
        self.word_embedding = nn.Embedding(src_vocab_size, embed_size)
        self.position_embedding = nn.Embedding(max_length, embed_size)

        self.layers = nn.ModuleList(
            [
                TransformerBlock(
                    embed_size,
                    heads,
                    dropout=dropout,
                    forward_expansion=forward_expansion,
                )
                for _ in range(num_layers)
            ]
        )

        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask, device):
        N, seq_length = x.shape
        positions = torch.arange(0, seq_length).expand(N, seq_length).to(device)
        out = self.dropout(
            (self.word_embedding(x) + self.position_embedding(positions))
        )

        # In the Encoder the query, key, value are all the same, it's in the
        # decoder this will change. This might look a bit odd in this case.
        for layer in self.layers:
            out = layer(out, out, out, mask)

        return out


class DecoderBlock(nn.Module):
    def __init__(self, embed_size, heads, forward_expansion, dropout):
        super(DecoderBlock, self).__init__()
        self.norm = nn.LayerNorm(embed_size)
        self.attention = SelfAttention(embed_size, heads=heads)
        self.transformer_block = TransformerBlock(
            embed_size, heads, dropout, forward_expansion
        )
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, value, key, src_mask, trg_mask):
        attention = self.attention(x, x, x, trg_mask)
        query = self.dropout(self.norm(attention + x))
        out = self.transformer_block(value, key, query, src_mask)
        return out


class Decoder(nn.Module):
    def __init__(
        self,
        trg_vocab_size,
        embed_size,
        num_layers,
        heads,
        forward_expansion,
        dropout,
        max_length,
    ):
        super(Decoder, self).__init__()
        self.word_embedding = nn.Embedding(trg_vocab_size, embed_size)
        self.position_embedding = nn.Embedding(max_length, embed_size)

        self.layers = nn.ModuleList(
            [
                DecoderBlock(embed_size, heads, forward_expansion, dropout)
                for _ in range(num_layers)
            ]
        )
        self.fc_out = nn.Linear(embed_size, trg_vocab_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, enc_out, src_mask, trg_mask, device):
        N, seq_length = x.shape
        positions = torch.arange(0, seq_length).expand(N, seq_length).to(device)
        x = self.dropout((self.word_embedding(x) + self.position_embedding(positions)))

        for layer in self.layers:
            x = layer(x, enc_out, enc_out, src_mask, trg_mask)

        out = self.fc_out(x)

        return out


class Transformer(nn.Module):
    def __init__(
        self,
        src_vocab_size,
        trg_vocab_size,
        src_pad_idx,
        trg_pad_idx,
        embed_size=512,
        num_layers=6,
        forward_expansion=4,
        heads=8,
        dropout=0,
        max_length=100,
    ):

        super(Transformer, self).__init__()

        self.encoder = Encoder(
            src_vocab_size,
            embed_size,
            num_layers,
            heads,
            forward_expansion,
            dropout,
            max_length,
        )

        self.decoder = Decoder(
            trg_vocab_size,
            embed_size,
            num_layers,
            heads,
            forward_expansion,
            dropout,
            max_length,
        )

        self.src_pad_idx = src_pad_idx
        self.trg_pad_idx = trg_pad_idx

    def make_src_mask(self, src, device):
        src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2)
        # (N, 1, 1, src_len)
        return src_mask.to(device)

    def make_trg_mask(self, trg, device):
        N, trg_len = trg.shape
        trg_mask = torch.tril(torch.ones((trg_len, trg_len))).expand(
            N, 1, trg_len, trg_len
        )

        return trg_mask.to(device)

    def forward(self, src, trg, device):
        src_mask = self.make_src_mask(src, device)
        trg_mask = self.make_trg_mask(trg, device)
        enc_src = self.encoder(src, src_mask, device)
        out = self.decoder(trg, enc_src, src_mask, trg_mask, device)
        return out

In [45]:
import pytorch_lightning as pl
import torch
import torch.nn as nn
import torch.nn.functional as F

class TransformerTranslateModel(pl.LightningModule):

    def __init__(self, hparams, *args, **kwargs):
        super().__init__()

        self.save_hyperparameters(hparams)

        print(f'Hparams =>\n\n{hparams}')

        self.model = Transformer(
            self.hparams.src_vocab_size,
            self.hparams.tgt_vocab_size,
            self.hparams.src_pad_idx,
            self.hparams.tgt_pad_idx,
            embed_size=self.hparams.embed_size,
            num_layers=self.hparams.num_layers,
            forward_expansion=self.hparams.forward_expansion,
            heads=self.hparams.heads,
            dropout=self.hparams.dropout,
            max_length=self.hparams.max_length,
        )

        self.loss = nn.CrossEntropyLoss(ignore_index=self.hparams.tgt_pad_idx)

    def forward(self, src_text, tgt_text):

        output = self.model(src_text, tgt_text, device=self.device)

        return output

    def shared_step(self, batch, batch_idx):
        src_text, tgt_text = batch

        # print(src_text.shape, tgt_text.shape)

        # remove <eos> from target and send
        logits = self(src_text, tgt_text[:, :-1])

        # reshape it into words * output_dim
        logits = logits.reshape(-1, logits.shape[2])
        # print(logits.shape)
        # we remove the <bos> from target
        target = tgt_text[:, 1:].reshape(-1)

        # print(target.shape)
        
        loss = self.loss(logits, target)

        metric = {'loss': loss}

        return metric


    def training_step(self, batch, batch_idx):
        metrics = self.shared_step(batch, batch_idx)

        log_metrics = {'train_loss': metrics['loss']}

        self.log_dict(log_metrics, prog_bar=True)

        return metrics


    def validation_step(self, batch, batch_idx):
        metrics = self.shared_step(batch, batch_idx)

        return metrics
    

    def validation_epoch_end(self, outputs):
        loss = torch.stack([x['loss'] for x in outputs]).mean()

        log_metrics = {'val_loss': loss}

        self.log_dict(log_metrics, prog_bar=True)

        return log_metrics


    def test_step(self, batch, batch_idx):
        return self.validation_step(batch, batch_idx)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.model.parameters(), lr=self.hparams.lr)
        return optimizer

In [36]:
ttc_translation = TTCTranslation(
    language_pair=('en', 'de'),
    spacy_language_pair=('en_core_web_sm', 'de_core_news_sm'),
    dataset='multi30k',
    batch_size=128
)

In [37]:
from omegaconf import OmegaConf

In [50]:
hparams = OmegaConf.create({
    'src_vocab_size': len(ttc_translation.vocab_transform[ttc_translation.src_lang]),
    'tgt_vocab_size': len(ttc_translation.vocab_transform[ttc_translation.tgt_lang]),
    'src_pad_idx': ttc_translation.PAD_IDX,
    'tgt_pad_idx': ttc_translation.PAD_IDX,
    'embed_size': 128,
    'num_layers': 2,
    'forward_expansion': 2,
    'heads': 2,
    'dropout': 0,
    'max_length': 10,
    'lr': 5e-4,
    'epochs': 5,
    'use_lr_finder': False
})

In [51]:
transformer = TransformerTranslateModel(hparams)

Hparams =>

{'src_vocab_size': 10838, 'tgt_vocab_size': 19215, 'src_pad_idx': 1, 'tgt_pad_idx': 1, 'embed_size': 128, 'num_layers': 2, 'forward_expansion': 2, 'heads': 2, 'dropout': 0, 'max_length': 10, 'lr': 0.0005, 'epochs': 5, 'use_lr_finder': False}


In [52]:
# trainer = pl.Trainer(gpus=0, max_epochs=hparams.epochs)

In [53]:
trainer = pl.Trainer(tpu_cores=8, max_epochs=hparams.epochs, checkpoint_callback=False)

GPU available: False, used: False
TPU available: True, using: 8 TPU cores
IPU available: False, using: 0 IPUs


In [1]:
trainer.fit(transformer, ttc_translation)

NameError: ignored

In [58]:
transformer.device

device(type='cpu')

In [ ]:
def translate_sentence(model, datamodule, sentence, device, max_length=20):
    # Load german tokenizer
    spacy_ger = spacy.load("de")

    # Create tokens using spacy and everything in lower case (which is what our vocab is)
    if type(sentence) == str:
        tokens = [token.text.lower() for token in spacy_ger(sentence)]
    else:
        tokens = [token.lower() for token in sentence]

    # Add <SOS> and <EOS> in beginning and end respectively
    tokens.insert(0, german.init_token)
    tokens.append(german.eos_token)

    # Go through each german token and convert to an index
    text_to_indices = [german.vocab.stoi[token] for token in tokens]

    # Convert to Tensor
    sentence_tensor = torch.LongTensor(text_to_indices).unsqueeze(1).to(device)

    outputs = [english.vocab.stoi["<sos>"]]
    for i in range(max_length):
        trg_tensor = torch.LongTensor(outputs).unsqueeze(1).to(device)

        with torch.no_grad():
            output = model(sentence_tensor, trg_tensor)

        best_guess = output.argmax(2)[-1, :].item()
        outputs.append(best_guess)

        if best_guess == english.vocab.stoi["<eos>"]:
            break

    translated_sentence = [english.vocab.itos[idx] for idx in outputs]
    # remove start token
    return translated_sentence[1:]

In [80]:
sentence = "hi how are you"

In [81]:
# Create tokens using spacy and everything in lower case (which is what our vocab is)
if type(sentence) == str:
    tokens = [token.lower() for token in ttc_translation.token_transform[ttc_translation.src_lang](sentence)]
else:
    tokens = [token.lower() for token in sentence]

# Add <SOS> and <EOS> in beginning and end respectively
tokens.insert(0, '<bos>')
tokens.append('<eos>')

# Go through each german token and convert to an index
text_to_indices = [ttc_translation.vocab_transform[ttc_translation.src_lang].get_stoi()[token] for token in tokens]

# Convert to Tensor
sentence_tensor = torch.LongTensor(text_to_indices).unsqueeze(0)

outputs = [ttc_translation.BOS_IDX]
for i in range(10):
    print(i)
    trg_tensor = torch.LongTensor(outputs).unsqueeze(0)

    with torch.no_grad():
        output = transformer(sentence_tensor, trg_tensor)

    best_guess = output.argmax(2)[:, -1].item()
    outputs.append(best_guess)

    if best_guess == ttc_translation.EOS_IDX:
        break

translated_sentence = [ttc_translation.vocab_transform[ttc_translation.tgt_lang].get_itos()[idx] for idx in outputs]

0
1
2
3
4
5
6
7
8
9


In [82]:
translated_sentence

['<bos>',
 'Theater-Schild',
 'Streitkräfte',
 'Mariachi-Kleidung',
 'nächtliches',
 'verzehr',
 'Halbnackter',
 'Bohnen',
 'Vögeln',
 'Zeitungsgeschäft',
 'operiert']

In [58]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

x = torch.tensor([[1, 5, 6, 4, 3, 9, 5, 2, 0], [1, 8, 7, 3, 4, 5, 6, 7, 2]]).to(
    device
)
trg = torch.tensor([[1, 7, 4, 3, 5, 9, 2, 0], [1, 5, 6, 2, 4, 7, 6, 2]]).to(device)

cpu


In [101]:
a, b = trg.shape

In [60]:
x.shape, trg.shape

(torch.Size([2, 9]), torch.Size([2, 8]))

In [61]:

src_pad_idx = 0
trg_pad_idx = 0
src_vocab_size = 10
trg_vocab_size = 19
model = Transformer(src_vocab_size, trg_vocab_size, src_pad_idx, trg_pad_idx, device=device).to(
    device
)


In [62]:
x

tensor([[1, 5, 6, 4, 3, 9, 5, 2, 0],
        [1, 8, 7, 3, 4, 5, 6, 7, 2]])

In [63]:
trg

tensor([[1, 7, 4, 3, 5, 9, 2, 0],
        [1, 5, 6, 2, 4, 7, 6, 2]])

In [64]:
trg[:, :-1]

tensor([[1, 7, 4, 3, 5, 9, 2],
        [1, 5, 6, 2, 4, 7, 6]])

In [65]:
out = model(x, trg)
print(out.shape)

torch.Size([2, 8, 19])


In [66]:
out = model(x, trg[:, :-1])
print(out.shape)

torch.Size([2, 7, 19])


In [67]:
out

tensor([[[ 0.2951,  0.1643, -0.2747,  0.3652, -0.1972, -0.8278,  0.4282,
           0.3892, -0.1439,  0.2059, -0.6906, -0.7613, -0.3035, -0.1787,
          -0.5844, -0.9108, -0.5637, -0.5992, -0.7560],
         [ 0.0180,  0.9289, -0.2546,  0.0580, -0.5211,  0.3094,  0.6083,
           0.1999,  1.1048,  0.8458,  0.0587, -1.0493,  0.5115,  0.4687,
          -0.3855, -0.3557, -0.1830, -0.7329, -1.3219],
         [ 0.3778,  0.6247,  0.4919, -0.0576,  0.0851, -0.0025,  0.8352,
          -0.0375,  0.5682, -0.0363, -0.3135, -0.6415,  1.0322,  0.2658,
          -0.5410, -0.6383,  0.0127, -0.4191, -0.2977],
         [-0.1501, -0.1214, -0.5211,  0.8485,  0.0799,  0.2861, -0.2637,
          -0.1062, -1.1467,  0.0139, -0.2851, -1.0000,  0.9866,  0.1749,
          -0.3502, -1.0612,  0.1819, -1.4339, -0.1691],
         [ 0.0605,  0.3795, -0.7087,  0.1393,  0.2491,  0.1029,  0.5625,
          -0.1687,  0.0632,  0.5568,  0.9216, -0.4791,  0.8981,  0.3617,
          -0.5308, -0.8235,  0.0753, -0.8778, 